In [ ]:
import pandas as pd
import tools as t

Предобработка тектов вакансий

In [ ]:
df_descr = pd.DataFrame(pd.read_json('deduplicated.json')[['id', 'description', 'key_skills']])

Удаление нерелевантных символов

In [ ]:
not_to_split = ['a/b', 'ci/cd', 'pl/sql', 'pl/pg', 'tcp/ip', 'ui/ux', 'ux/ui', 'а/б', 'а/в', 'в/из']

In [ ]:
df_descr['d_clean'] = df_descr['description'].apply(lambda x: t.clean_text(x, not_to_split))
df_descr['ks_clean'] = df_descr['key_skills'].apply(lambda x: t.clean_text(str(x), not_to_split))

Токенизация

In [ ]:
from nltk import word_tokenize

In [ ]:
df_descr['d_tokens'] = df_descr['d_clean'].apply(lambda x: word_tokenize(x, language = 'russian'))
df_descr['ks_tokens'] = df_descr['ks_clean'].apply(lambda x: word_tokenize(x, language = 'russian'))

Лемматизация (23771 текста обрабатывались 55 мин 29 сек)

In [ ]:
df_descr['d_lemmatized'] = df_descr['d_tokens'].apply(lambda x: t.lemmatization(x))

In [ ]:
df_descr['ks_lemmatized'] = df_descr['ks_tokens'].apply(lambda x: t.lemmatization(x))

In [ ]:
df_descr['ks_clean'][df_descr['ks_clean'].str.contains('excel') == True]

Скачивание стоп-слов для русского языка и добавление новых стоп-слов

In [ ]:
from nltk.corpus import stopwords
sw_to_add = ['хорошее', 'знание', 'знать', 'опыт', 'работы', 'организация', 'работать', 'ms', 'условия', 'умение', 'навык']
sw_to_add = t.lemmatization(sw_to_add)
stopwords = stopwords.words('russian')
stopwords = stopwords + sw_to_add

Удаление стоп-слов

In [ ]:
df_descr['d_wo_sw'] = df_descr['d_lemmatized'].apply(lambda x: t.remove_stop_words(x, stopwords))
df_descr['ks_wo_sw'] = df_descr['ks_lemmatized'].apply(lambda x: t.remove_stop_words(x, stopwords))

In [ ]:
df_full = pd.DataFrame(pd.read_json('deduplicated.json'))

In [ ]:
df_descr['proffesional_roles'] = df_full['proffesional_roles']

In [ ]:
df_descr.to_json('preprocessed_text.json')

In [ ]:
df_descr = pd.read_json('preprocessed_text.json')

In [ ]:
df_descr['d_full'] = df_descr['d_wo_sw'].apply(lambda x: ' '.join(x))
df_descr['ks_full'] = df_descr['ks_wo_sw'].apply(lambda x: ' '.join(x))

SKILL COUNT BASED ON DICTIONARY

Создаем датафрейм из id, полного описания вакансии и key_skills, добавляем столбец с пустыми списками (для полученных навыков) 

In [ ]:
descr = pd.DataFrame({'id': df_descr['id'], 'd_full': df_descr['d_full'], 'ks_full': df_descr['ks_full']})
descr['extracted_skills'] = descr.apply(lambda _: [], axis = 1)

Создаем словарь навыков

In [ ]:
df_dict = pd.read_excel('automatic generated dict.xlsx')['name'].to_list()
skills = dict.fromkeys(df_dict, 0)

Функция для подсчета частоты встечаемости навыков и вытаскивания навыков в отдельное поле

In [ ]:
import regex as re

In [ ]:
def skills_count(id):
    ind = descr.index[descr['id'] == id].tolist()[0]
    text = descr['d_full'][ind]
    ks = descr['ks_full'][ind]
    for skill in skills:
        if re.search(skill, ks) != None:
            descr['extracted_skills'][ind].append(skill)
            skills[skill] += 1
        elif re.search(skill, text) != None:
            descr['extracted_skills'][ind].append(skill)
            skills[skill] += 1

In [ ]:
descr['id'].apply(lambda x: skills_count(x))

Сортировка

In [ ]:
skills = dict(sorted(skills.items(), key=lambda item: item[1], reverse=True))

In [ ]:
skills

Сохранение данных

In [ ]:
descr.to_json('extracted_skills.json')